In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

dataset_name = "music4all"
dataset_relative_location = "../../dataset"

dataset_path = Path(dataset_relative_location) / dataset_name

datasets = {}

csvsPattern = "*.csv"
csvs = list(dataset_path.glob(csvsPattern))
csvs

for csv in csvs:
    datasets[csv.stem] = pd.read_csv(csv, delimiter="\t")

# Taking a look at the datasets we have
datasets.keys()

dict_keys(['id_tags', 'id_metadata', 'id_information', 'listening_history', 'id_lang', 'id_genres'])

In [19]:
listening_history = datasets["listening_history"].copy(deep=True)
listening_history

,user,song,timestamp
0,user_007XIjOr,DaTQ53TUmfP93FSr,2019-02-20 12:28
1,user_007XIjOr,dGeyvi5WCOjDU7da,2019-02-20 12:35
2,user_007XIjOr,qUm54NYOjeFhmKYx,2019-02-20 12:48
3,user_007XIjOr,FtnuMT1DlevSR2n5,2019-02-20 12:52
4,user_007XIjOr,LHETTZcSZLeaVOGh,2019-02-20 13:09
...,...,...,...
5109587,user_zzWscYTy,BBiswLufo26YQCT7,2019-01-10 15:57
5109588,user_zzWscYTy,5ZHgff3sjETIiedr,2019-01-10 16:21
5109589,user_zzWscYTy,m4O1iLh6fC43xjRy,2019-01-10 16:48
5109590,user_zzWscYTy,mvUaP8k67qOFfA65,2019-01-10 21:13


In [9]:
import pickle
# Loading disruption index:

with open('dataset/disruption_index/transfer-learning-disruption-index-gamma-0-1.pickle', 'rb') as handle:
    disruption_index_dict = pickle.load(handle)

In [14]:

from collections import OrderedDict

def sort_disruption_dict(disruption_dict):
    # [ni, nj, nk, float((ni-nj)) / float((ni+nj+nk))]
    sorting = sorted(disruption_index_dict.items(), key=lambda kv: (kv[1][3], kv[1][0]), reverse=True)
    return OrderedDict(sorting)

ordered_disruption_dict = sort_disruption_dict(disruption_index_dict)

In [20]:


for index, row in listening_history.iterrows():
    print(row['user'], row['song'])
    break

ordered_disruption_dict["7DQI6hI5KiggQHkB"]

user_007XIjOr DaTQ53TUmfP93FSr


[657, 0, 0, 1.0]

In [34]:
grouped_users = listening_history.groupby("user")

user_set = set()

# Check for users that listened at least one of the songs in the partitioned dataset
for user, songs_listened_by_user in grouped_users:
    for _, row in songs_listened_by_user.iterrows():
        if row["song"] in ordered_disruption_dict:
            user_set.add(user)
            break

In [42]:
len(user_set)

14113

In [41]:
len(pd.unique(listening_history["user"]))

0.9990089898775395